In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data



# Fetching Data

In [ ]:
def fetch_historical_data(start_date, end_date, symbols):
    df_prices = data.DataReader(symbols, start=start_date, end=end_date, data_source='yahoo')
    df_prices = df_prices[['Adj Close']]
    df_prices.columns = [' '.join(col).strip() for col in df_prices.columns.values]
    return df_prices

# Calculating Metrices

In [ ]:
def calculate_portfolio_weights(symbols, portfolio_value, price_at_end_date):
    portfolio_weights_random = np.random.dirichlet(np.ones(len(symbols)), size=1)[0]
    no_of_shares = [round((x * portfolio_value) / price, 5) for x,
    price in zip(portfolio_weights_random, price_at_end_date)]
    return portfolio_weights_random, no_of_shares

def calculate_metrics(df_returns, df_mean_stdev, risk_free_rate):
    mu = np.array(df_mean_stdev["Mean Log Daily Return"].values.tolist())
    sigma = np.array(df_mean_stdev["StdDev Log Daily Return"].values.tolist())

    iteration_std_dev = df_mean_stdev.tail(1).values[0][2]
    iteration_mean = df_mean_stdev.tail(1).values[0][1]

    negative_returns_only = df_returns[df_returns['Log Daily Returns Adj Close Portfolio'] < 0]
    iteration_negative_returns_std_dev = negative_returns_only['Log Daily Returns Adj Close Portfolio'].std()

    iteration_sharpe_ratio = round(((np.exp(iteration_mean) - 1) - risk_free_rate) / (np.exp(iteration_std_dev) - 1), 3)
    iteration_sortino_ratio = round(((np.exp(iteration_mean) - 1) - risk_free_rate) / (np.exp(iteration_negative_returns_std_dev) - 1), 3)

    return iteration_sharpe_ratio, iteration_sortino_ratio, iteration_std_dev, iteration_mean


# Monte Carlo Simulations

In [ ]:
def efficient_portfolio_historical(start_date, end_date, symbols, portfolio_value, no_of_iterations_mc, annual_risk_free_rate, image_counter, target_folder):
    risk_free_rate = (1 + annual_risk_free_rate) ** (1/252) - 1
    
    df_prices = fetch_historical_data(start_date, end_date, symbols)
    price_at_end_date = [df_prices[[f'Adj Close {symbol}']][-1:].values[0][0] for symbol in symbols]
    
    results_table = []

    for _ in range(no_of_iterations_mc):
        portfolio_weights_random, no_of_shares = calculate_portfolio_weights(symbols, portfolio_value, price_at_end_date)

        df_prices_inner = df_prices.copy()
        df_prices_inner["Adj Close Portfolio"] = df_prices_inner.mul(no_of_shares).sum(1)

        df_returns, df_mean_stdev = calc_returns(df_prices_inner, symbols)

        iteration_sharpe_ratio, iteration_sortino_ratio, iteration_std_dev, iteration_mean = calculate_metrics(df_returns, df_mean_stdev, risk_free_rate)

        results_table.append([portfolio_weights_random, iteration_std_dev, iteration_mean, iteration_sharpe_ratio, iteration_sortino_ratio])

        df_prices_inner.drop('Adj Close Portfolio', inplace=True, axis=1)

    final_results_table = pd.DataFrame(results_table, columns=["Weights", "Std Dev", "Mean", "Sharpe Ratio", "Sortino Ratio"])

    historical_df_returns, historical_df_mean_stdev = calc_returns(df_prices, symbols)
    historical_df_mean_stdev = historical_df_mean_stdev[['Stock', 'StdDev Log Daily Return', 'Mean Log Daily Return']]
    historical_df_mean_stdev.columns = ['Stock', 'Std Dev', 'Mean']

    fig, ax = plt.subplots(figsize=(10, 5))
    final_results_table.plot.scatter(x="Std Dev", y='Mean', ax=ax)
    historical_df_mean_stdev.plot.scatter(x="Std Dev", y='Mean', c='r', marker='x', ax=ax)

    sharpe_std_dev = final_results_table.nlargest(1, ['Sharpe Ratio'])['Std Dev'].values[0]
    sharpe_mean = final_results_table.nlargest(1, ['Sharpe Ratio'])['Mean'].values[0]
    sharperounded_weights = [round(num, 4) for num in final_results_table.nlargest(1, ['Sharpe Ratio'])['Weights'].values[0]]
    sharpeweight_string = [[symbol, weight] for symbol, weight in zip(symbols, sharperounded_weights)]
    sharpe_label = "Optimal Sharpe Ratio"
    sharpe_detail = f'Optimal Sharpe Ratio: {final_results_table.nlargest(1, ["Sharpe Ratio"])["Sharpe Ratio"].values[0]} with Weights {sharpeweight_string}'

    sortino_std_dev = final_results_table.nlargest(1, ['Sortino Ratio'])['Std Dev'].values[0]
    sortino_mean = final_results_table.nlargest(1, ['Sortino Ratio'])['Mean'].values[0]
    sortinorounded_weights = [round(num, 4) for num in final_results_table.nlargest(1, ['Sortino Ratio'])['Weights'].values[0]]
    sortinoweight_string = [[symbol, weight] for symbol, weight in zip(symbols, sortinorounded_weights)]
    sortino_label = 'Optimal Sortino Ratio'
    sortino_detail = f'Optimal Sortino Ratio: {final_results_table.nlargest(1, ["Sortino Ratio"])["Sortino Ratio"].values[0]} with Weights {sortinoweight_string}'

    sharpe_sortino = pd.DataFrame(zip([sharpe_std_dev, sortino_std_dev], [sharpe_mean, sortino_mean]), index=['Optimal Sharpe', 'Optimal Sortino'], columns=['Std Dev', 'Mean'])
    sharpe_sortino.plot.scatter(x="Std Dev", y='Mean', c='g', marker='x', ax=ax)

    txt = list(historical_df_mean_stdev['Stock']) + [sharpe_label, sortino_label]
    z = list(historical_df_mean_stdev['Std Dev']) + [sharpe_std_dev, sortino_std_dev]
    y = list(historical_df_mean_stdev['Mean']) + [sharpe_mean, sortino_mean]
    for i, text in enumerate(txt):
        ax.annotate(text, (z[i], y[i]))

    plt.title(f"Mean vs Std Dev Of Log Returns For {no_of_iterations_mc} Different Portfolio Weights")
    plt.savefig(f'static/{target_folder}/{image_counter}_efficientportfolio.png')
    print(sharpe_detail)
    print(sortino_detail)

    final_results_table['Log Returns Std Dev'] = final_results_table['Std Dev']
    final_results_table['Log Returns Mean'] = final_results_table['Mean']
    final_results_table = final_results_table[['Weights', 'Log Returns Std Dev', 'Log Returns Mean', 'Sharpe Ratio', 'Sortino Ratio']]

    return final_results